# **Copy Data from Drive to Colab Hard**

In [ ]:
# Mount drive

In [ ]:
!cp /content/drive/MyDrive/Masoudi/dataset.zip .

In [ ]:
!unzip -qq dataset.zip

In [1]:
!cp /content/drive/MyDrive/Masoudi/4.zip .
!unzip -qq /content/4.zip

In [3]:
!unzip -qq 1.zip -d  /content/example/test_data

In [4]:
!unzip -qq 1.zip -d  /content/example/training_data

# **helper_code.py**

In [5]:
import os, numpy as np, scipy as sp, scipy.io

### Challenge data I/O functions

# Find the folders with data files.
def find_data_folders(root_folder):
    data_folders = list()
    for x in os.listdir(root_folder):
        data_folder = os.path.join(root_folder, x)
        if os.path.isdir(data_folder):
            data_folders.append(x)
    return sorted(data_folders)

def load_challenge_data(data_folder, patient_id):
    # Define file location.
    patient_metadata_file = os.path.join(data_folder, patient_id, patient_id + '.txt')
    recording_metadata_file = os.path.join(data_folder, patient_id, patient_id + '.tsv')

    # Load non-recording data.
    patient_metadata = load_text_file(patient_metadata_file)
    recording_metadata = load_text_file(recording_metadata_file)

    # Load recordings.
    recordings = list()
    recording_ids = get_recording_ids(recording_metadata)
    for recording_id in recording_ids:
        if recording_id != 'nan':
            recording_location = os.path.join(data_folder, patient_id, recording_id)
            recording_data, sampling_frequency, channels = load_recording(recording_location)
        else:
            recording_data = None
            sampling_frequency = None
            channels = None
        recordings.append((recording_data, sampling_frequency, channels))

    return patient_metadata, recording_metadata, recordings

# Load the WFDB data for the Challenge (but not all possible WFDB files).
def load_recording(record_name):
    # Allow either the record name or the header filename.
    root, ext = os.path.splitext(record_name)
    if ext=='':
        header_file = record_name + '.hea'
    else:
        header_file = record_name

    # Load the header file.
    if not os.path.isfile(header_file):
        raise FileNotFoundError('{} recording not found.'.format(record_name))

    with open(header_file, 'r') as f:
        header = [l.strip() for l in f.readlines() if l.strip()]

    # Parse the header file.
    record_name = None
    num_signals = None
    sampling_frequency = None
    num_samples = None
    signal_files = list()
    gains = list()
    offsets = list()
    channels = list()
    initial_values = list()
    checksums = list()

    for i, l in enumerate(header):
        arrs = [arr.strip() for arr in l.split(' ')]
        # Parse the record line.
        if i==0:
            record_name = arrs[0]
            num_signals = int(arrs[1])
            sampling_frequency = float(arrs[2])
            num_samples = int(arrs[3])
        # Parse the signal specification lines.
        else:
            signal_file = arrs[0]
            gain = float(arrs[2].split('/')[0])
            offset = int(arrs[4])
            initial_value = int(arrs[5])
            checksum = int(arrs[6])
            channel = arrs[8]
            signal_files.append(signal_file)
            gains.append(gain)
            offsets.append(offset)
            initial_values.append(initial_value)
            checksums.append(checksum)
            channels.append(channel)

    # Check that the header file only references one signal file. WFDB format  allows for multiple signal files, but we have not
    # implemented that here for simplicity.
    num_signal_files = len(set(signal_files))
    if num_signal_files!=1:
        raise NotImplementedError('The header file {}'.format(header_file) \
            + ' references {} signal files; one signal file expected.'.format(num_signal_files))

    # Load the signal file.
    head, tail = os.path.split(header_file)
    signal_file = os.path.join(head, list(signal_files)[0])
    data = np.asarray(sp.io.loadmat(signal_file)['val'])

    # Check that the dimensions of the signal data in the signal file is consistent with the dimensions for the signal data given
    # in the header file.
    num_channels = len(channels)
    if np.shape(data)!=(num_channels, num_samples):
        raise ValueError('The header file {}'.format(header_file) \
            + ' is inconsistent with the dimensions of the signal file.')

    # Check that the initial value and checksums for the signal data in the signal file are consistent with the initial value and
    # checksums for the signal data given in the header file.
    for i in range(num_channels):
        if data[i, 0]!=initial_values[i]:
            raise ValueError('The initial value in header file {}'.format(header_file) \
                + ' is inconsistent with the initial value for channel'.format(channels[i]))
        if np.sum(data[i, :])!=checksums[i]:
            raise ValueError('The checksum in header file {}'.format(header_file) \
                + ' is inconsistent with the initial value for channel'.format(channels[i]))

    # Rescale the signal data using the ADC gains and ADC offsets.
    rescaled_data = np.zeros(np.shape(data), dtype=np.float32)
    for i in range(num_channels):
        rescaled_data[i, :] = (data[i, :]-offsets[i])/gains[i]

    return rescaled_data, sampling_frequency, channels

# Reorder/reselect the channels.
def reorder_recording_channels(current_data, current_channels, reordered_channels):
    if current_channels == reordered_channels:
        return current_data
    else:
        indices = list()
        for channel in reordered_channels:
            if channel in current_channels:
                i = current_channels.index(channel)
                indices.append(i)
        num_channels = len(reordered_channels)
        num_samples = np.shape(current_data)[1]
        reordered_data = np.zeros((num_channels, num_samples))
        reordered_data[:, :] = current_data[indices, :]
        return reordered_data

### Helper Challenge data I/O functions

# Load text file as a string.
def load_text_file(filename):
    with open(filename, 'r') as f:
        data = f.read()
    return data

# Get a variable from the patient metadata.
def get_variable(text, variable_name, variable_type, preserve_nan=True):
    variable = None
    for l in text.split('\n'):
        if l.startswith(variable_name):
            variable = l.split(':')[1].strip()
            if preserve_nan and variable.lower() == 'nan':
                variable = float('nan')
            else:
                variable = variable_type(variable)
            return variable

# Get a column from the recording metadata.
def get_column(string, column, variable_type, sep='\t'):
    variables = list()
    for i, l in enumerate(string.split('\n')):
        arrs = [arr.strip() for arr in l.split(sep) if arr.strip()]
        if i==0:
            column_index = arrs.index(column)
        elif arrs:
            variable = arrs[column_index]
            variable = variable_type(variable)
            variables.append(variable)
    return np.asarray(variables)

# Get the patient ID variable from the patient data.
def get_patient_id(string):
    return get_variable(string, 'Patient', str)

# Get the age variable (in years) from the patient data.
def get_age(string):
    return get_variable(string, 'Age', int)

# Get the sex variable from the patient data.
def get_sex(string):
    return get_variable(string, 'Sex', str)

# Get the ROSC variable (in minutes) from the patient data.
def get_rosc(string):
    return get_variable(string, 'ROSC', int)

# Get the OHCA variable from the patient data.
def get_ohca(string):
    return get_variable(string, 'OHCA', bool)

# Get the VFib variable from the patient data.
def get_vfib(string):
    return get_variable(string, 'VFib', bool)

# Get the TTM variable (in Celsius) from the patient data.
def get_ttm(string):
    return get_variable(string, 'TTM', int)

# Get the Outcome variable from the patient data.
def get_outcome(string):
    variable = get_variable(string, 'Outcome', str)
    if variable is None or is_nan(variable):
        raise ValueError('No outcome available. Is your code trying to load labels from the hidden data?')
    if variable == 'Good':
        variable = 0
    elif variable == 'Poor':
        variable = 1
    return variable

# Get the Outcome probability variable from the patient data.
def get_outcome_probability(string):
    variable = sanitize_scalar_value(get_variable(string, 'Outcome probability', str))
    if variable is None or is_nan(variable):
        raise ValueError('No outcome available. Is your code trying to load labels from the hidden data?')
    return variable

# Get the CPC variable from the patient data.
def get_cpc(string):
    variable = sanitize_scalar_value(get_variable(string, 'CPC', str))
    if variable is None or is_nan(variable):
        raise ValueError('No CPC score available. Is your code trying to load labels from the hidden data?')
    return variable

# Get the hour number column from the patient data.
def get_hours(string):
    return get_column(string, 'Hour', int)

# Get the time column from the patient data.
def get_times(string):
    return get_column(string, 'Time', str)

# Get the quality score column from the patient data.
def get_quality_scores(string):
    return get_column(string, 'Quality', float)

# Get the recording IDs column from the patient data.
def get_recording_ids(string):
    return get_column(string, 'Record', str)

### Challenge label and output I/O functions

# Load the Challenge labels for one file.
def load_challenge_label(string):
    if os.path.isfile(string):
        string = load_text_file(string)

    outcome = get_outcome(string)
    cpc = get_cpc(string)

    return outcome, cpc

# Load the Challenge labels for all of the files in a folder.
def load_challenge_labels(folder):
    patient_folders = find_data_folders(folder)
    num_patients = len(patient_folders)

    patient_ids = list()
    outcomes = np.zeros(num_patients, dtype=np.bool_)
    cpcs = np.zeros(num_patients, dtype=np.float64)

    for i in range(num_patients):
        patient_metadata_file = os.path.join(folder, patient_folders[i], patient_folders[i] + '.txt')
        patient_metadata = load_text_file(patient_metadata_file)

        patient_ids.append(get_patient_id(patient_metadata))
        outcomes[i] = get_outcome(patient_metadata)
        cpcs[i] = get_cpc(patient_metadata)

    return patient_ids, outcomes, cpcs

# Save the Challenge outputs for one file.
def save_challenge_outputs(filename, patient_id, outcome, outcome_probability, cpc):
    # Sanitize values, e.g., in case they are a singleton array.
    outcome = sanitize_boolean_value(outcome)
    outcome_probability = sanitize_scalar_value(outcome_probability)
    cpc = sanitize_scalar_value(cpc)

    # Format Challenge outputs.
    patient_string = 'Patient: {}'.format(patient_id)
    if outcome == 0:
        outcome = 'Good'
    elif outcome == 1:
        outcome = 'Poor'
    outcome_string = 'Outcome: {}'.format(outcome)
    outcome_probability_string = 'Outcome probability: {:.3f}'.format(float(outcome_probability))
    cpc_string = 'CPC: {:.3f}'.format(int(float(cpc)) if is_integer(cpc) else float(cpc))
    output_string = patient_string + '\n' + \
        outcome_string + '\n' + outcome_probability_string + '\n' + cpc_string + '\n'

    # Write the Challenge outputs.
    if filename is not None:
        with open(filename, 'w') as f:
            f.write(output_string)

    return output_string

# Load the Challenge outputs for one file.
def load_challenge_output(string):
    if os.path.isfile(string):
        string = load_text_file(string)

    patient_id = get_patient_id(string)
    outcome = get_outcome(string)
    outcome_probability = get_outcome_probability(string)
    cpc = get_cpc(string)

    return patient_id, outcome, outcome_probability, cpc

# Load the Challenge outputs for all of the files in folder.
def load_challenge_outputs(folder, patient_ids):
    num_patients = len(patient_ids)
    outcomes = np.zeros(num_patients, dtype=np.bool_)
    outcome_probabilities = np.zeros(num_patients, dtype=np.float64)
    cpcs = np.zeros(num_patients, dtype=np.float64)

    for i in range(num_patients):
        output_file = os.path.join(folder, patient_ids[i], patient_ids[i] + '.txt')
        patient_id, outcome, outcome_probability, cpc = load_challenge_output(output_file)
        outcomes[i] = outcome
        outcome_probabilities[i] = outcome_probability
        cpcs[i] = cpc

    return outcomes, outcome_probabilities, cpcs

### Other helper functions

# Check if a variable is a number or represents a number.
def is_number(x):
    try:
        float(x)
        return True
    except (ValueError, TypeError):
        return False

# Check if a variable is an integer or represents an integer.
def is_integer(x):
    if is_number(x):
        return float(x).is_integer()
    else:
        return False

# Check if a variable is a finite number or represents a finite number.
def is_finite_number(x):
    if is_number(x):
        return np.isfinite(float(x))
    else:
        return False

# Check if a variable is a NaN (not a number) or represents a NaN.
def is_nan(x):
    if is_number(x):
        return np.isnan(float(x))
    else:
        return False

# Remove any quotes, brackets (for singleton arrays), and/or invisible characters.
def remove_extra_characters(x):
    return str(x).replace('"', '').replace("'", "").replace('[', '').replace(']', '').replace(' ', '').strip()

# Sanitize boolean values, e.g., from the Challenge outputs.
def sanitize_boolean_value(x):
    x = remove_extra_characters(x)
    if (is_finite_number(x) and float(x)==0) or (x in ('False', 'false', 'F', 'f')):
        return 0
    elif (is_finite_number(x) and float(x)==1) or (x in ('True', 'true', 'T', 't')):
        return 1
    else:
        return float('nan')

# Santize scalar values, e.g., from the Challenge outputs.
def sanitize_scalar_value(x):
    x = remove_extra_characters(x)
    if is_number(x):
        return float(x)
    else:
        return float('nan')

evaluation

In [6]:
import os, os.path, sys, numpy as np

# Evaluate the models.
def evaluate_model(label_folder, output_folder):
    # Load labels and model outputs.
    patient_ids, label_outcomes, label_cpcs = load_challenge_labels(label_folder)
    output_outcomes, output_outcome_probabilities, output_cpcs = load_challenge_outputs(output_folder, patient_ids)

    # Evaluate the models.
    challenge_score = compute_challenge_score(label_outcomes, output_outcome_probabilities)
    auroc_outcomes, auprc_outcomes = compute_auc(label_outcomes, output_outcome_probabilities)
    accuracy_outcomes, _, _ = compute_accuracy(label_outcomes, output_outcomes)
    f_measure_outcomes, _, _ = compute_f_measure(label_outcomes, output_outcomes)

    mse_cpcs = compute_mse(label_cpcs, output_cpcs)
    mae_cpcs = compute_mae(label_cpcs, output_cpcs)

    # Return the results.
    return challenge_score, auroc_outcomes, auprc_outcomes, accuracy_outcomes, f_measure_outcomes, mse_cpcs, mae_cpcs

# Compute the Challenge score.
def compute_challenge_score(labels, outputs):
    assert len(labels) == len(outputs)
    num_instances = len(labels)

    # Use the unique output values as the thresholds for the positive and negative classes.
    thresholds = np.unique(outputs)
    thresholds = np.append(thresholds, thresholds[-1]+1)
    thresholds = thresholds[::-1]
    num_thresholds = len(thresholds)

    idx = np.argsort(outputs)[::-1]

    # Initialize the TPs, FPs, FNs, and TNs with no positive outputs.
    tp = np.zeros(num_thresholds)
    fp = np.zeros(num_thresholds)
    fn = np.zeros(num_thresholds)
    tn = np.zeros(num_thresholds)

    tp[0] = 0
    fp[0] = 0
    fn[0] = np.sum(labels == 1)
    tn[0] = np.sum(labels == 0)

    # Update the TPs, FPs, FNs, and TNs using the values at the previous threshold.
    i = 0
    for j in range(1, num_thresholds):
        tp[j] = tp[j-1]
        fp[j] = fp[j-1]
        fn[j] = fn[j-1]
        tn[j] = tn[j-1]

        while i < num_instances and outputs[idx[i]] >= thresholds[j]:
            if labels[idx[i]]:
                tp[j] += 1
                fn[j] -= 1
            else:
                fp[j] += 1
                tn[j] -= 1
            i += 1

    # Compute the TPRs and FPRs.
    tpr = np.zeros(num_thresholds)
    fpr = np.zeros(num_thresholds)
    for j in range(num_thresholds):
        if tp[j] + fn[j] > 0:
            tpr[j] = float(tp[j]) / float(tp[j] + fn[j])
            fpr[j] = float(fp[j]) / float(fp[j] + tn[j])
        else:
            tpr[j] = float('nan')
            fpr[j] = float('nan')

    # Find the largest TPR such that FPR <= 0.05.
    max_fpr = 0.05
    max_tpr = float('nan')
    if np.any(fpr <= max_fpr):
        indices = np.where(fpr <= max_fpr)
        max_tpr = np.max(tpr[indices])

    return max_tpr

# Compute area under the receiver operating characteristic curve (AUROC) and area under the precision recall curve (AUPRC).
def compute_auc(labels, outputs):
    assert len(labels) == len(outputs)
    num_instances = len(labels)

    # Use the unique output values as the thresholds for the positive and negative classes.
    thresholds = np.unique(outputs)
    thresholds = np.append(thresholds, thresholds[-1]+1)
    thresholds = thresholds[::-1]
    num_thresholds = len(thresholds)

    idx = np.argsort(outputs)[::-1]

    # Initialize the TPs, FPs, FNs, and TNs with no positive outputs.
    tp = np.zeros(num_thresholds)
    fp = np.zeros(num_thresholds)
    fn = np.zeros(num_thresholds)
    tn = np.zeros(num_thresholds)

    tp[0] = 0
    fp[0] = 0
    fn[0] = np.sum(labels == 1)
    tn[0] = np.sum(labels == 0)

    # Update the TPs, FPs, FNs, and TNs using the values at the previous threshold.
    i = 0
    for j in range(1, num_thresholds):
        tp[j] = tp[j-1]
        fp[j] = fp[j-1]
        fn[j] = fn[j-1]
        tn[j] = tn[j-1]

        while i < num_instances and outputs[idx[i]] >= thresholds[j]:
            if labels[idx[i]]:
                tp[j] += 1
                fn[j] -= 1
            else:
                fp[j] += 1
                tn[j] -= 1
            i += 1

    # Compute the TPRs, TNRs, and PPVs at each threshold.
    tpr = np.zeros(num_thresholds)
    tnr = np.zeros(num_thresholds)
    ppv = np.zeros(num_thresholds)
    for j in range(num_thresholds):
        if tp[j] + fn[j]:
            tpr[j] = float(tp[j]) / float(tp[j] + fn[j])
        else:
            tpr[j] = float('nan')
        if fp[j] + tn[j]:
            tnr[j] = float(tn[j]) / float(fp[j] + tn[j])
        else:
            tnr[j] = float('nan')
        if tp[j] + fp[j]:
            ppv[j] = float(tp[j]) / float(tp[j] + fp[j])
        else:
            ppv[j] = float('nan')

    # Compute AUROC as the area under a piecewise linear function with TPR/sensitivity (x-axis) and TNR/specificity (y-axis) and
    # AUPRC as the area under a piecewise constant with TPR/recall (x-axis) and PPV/precision (y-axis).
    auroc = 0.0
    auprc = 0.0
    for j in range(num_thresholds-1):
        auroc += 0.5 * (tpr[j+1] - tpr[j]) * (tnr[j+1] + tnr[j])
        auprc += (tpr[j+1] - tpr[j]) * ppv[j+1]

    return auroc, auprc

# Construct the one-hot encoding of data for the given classes.
def compute_one_hot_encoding(data, classes):
    num_instances = len(data)
    num_classes = len(classes)

    one_hot_encoding = np.zeros((num_instances, num_classes), dtype=np.bool_)
    unencoded_data = list()
    for i, x in enumerate(data):
        for j, y in enumerate(classes):
            if (x == y) or (is_nan(x) and is_nan(y)):
                one_hot_encoding[i, j] = 1

    return one_hot_encoding

# Compute the binary confusion matrix, where the columns are the expert labels and the rows are the classifier labels for the given
# classes.
def compute_confusion_matrix(labels, outputs, classes):
    assert np.shape(labels) == np.shape(outputs)

    num_instances = len(labels)
    num_classes = len(classes)

    A = np.zeros((num_classes, num_classes))
    for k in range(num_instances):
        for i in range(num_classes):
            for j in range(num_classes):
                if outputs[k, i] == 1 and labels[k, j] == 1:
                    A[i, j] += 1

    return A

# Construct the binary one-vs-rest confusion matrices, where the columns are the expert labels and the rows are the classifier
# for the given classes.
def compute_one_vs_rest_confusion_matrix(labels, outputs, classes):
    assert np.shape(labels) == np.shape(outputs)

    num_instances = len(labels)
    num_classes = len(classes)

    A = np.zeros((num_classes, 2, 2))
    for i in range(num_instances):
        for j in range(num_classes):
            if labels[i, j] == 1 and outputs[i, j] == 1: # TP
                A[j, 0, 0] += 1
            elif labels[i, j] == 0 and outputs[i, j] == 1: # FP
                A[j, 0, 1] += 1
            elif labels[i, j] == 1 and outputs[i, j] == 0: # FN
                A[j, 1, 0] += 1
            elif labels[i, j] == 0 and outputs[i, j] == 0: # TN
                A[j, 1, 1] += 1

    return A

# Compute accuracy.
def compute_accuracy(labels, outputs):
    # Compute the confusion matrix.
    classes = np.unique(np.concatenate((labels, outputs)))
    labels = compute_one_hot_encoding(labels, classes)
    outputs = compute_one_hot_encoding(outputs, classes)
    A = compute_confusion_matrix(labels, outputs, classes)

    # Compute accuracy.
    if np.sum(A) > 0:
        accuracy = np.trace(A) / np.sum(A)
    else:
        accuracy = float('nan')

    # Compute per-class accuracy.
    num_classes = len(classes)
    per_class_accuracy = np.zeros(num_classes)
    for i in range(num_classes):
        if np.sum(labels[:, i]) > 0:
            per_class_accuracy[i] = A[i, i] / np.sum(A[:, i])
        else:
            per_class_accuracy[i] = float('nan')

    return accuracy, per_class_accuracy, classes

# Compute macro F-measure.
def compute_f_measure(labels, outputs):
    # Compute confusion matrix.
    classes = np.unique(np.concatenate((labels, outputs)))
    labels = compute_one_hot_encoding(labels, classes)
    outputs = compute_one_hot_encoding(outputs, classes)
    A = compute_one_vs_rest_confusion_matrix(labels, outputs, classes)

    num_classes = len(classes)
    per_class_f_measure = np.zeros(num_classes)
    for k in range(num_classes):
        tp, fp, fn, tn = A[k, 0, 0], A[k, 0, 1], A[k, 1, 0], A[k, 1, 1]
        if 2 * tp + fp + fn > 0:
            per_class_f_measure[k] = float(2 * tp) / float(2 * tp + fp + fn)
        else:
            per_class_f_measure[k] = float('nan')

    if np.any(np.isfinite(per_class_f_measure)):
        macro_f_measure = np.nanmean(per_class_f_measure)
    else:
        macro_f_measure = float('nan')

    return macro_f_measure, per_class_f_measure, classes

# Compute mean-squared error.
def compute_mse(labels, outputs):
    assert len(labels) == len(outputs)

    labels = np.asarray(labels, dtype=np.float64)
    outputs = np.asarray(outputs, dtype=np.float64)
    mse = np.mean((labels - outputs)**2)

    return mse

# Compute mean-absolute error.
def compute_mae(labels, outputs):
    assert len(labels) == len(outputs)

    labels = np.asarray(labels, dtype=np.float64)
    outputs = np.asarray(outputs, dtype=np.float64)
    mae = np.mean(np.abs(labels - outputs))

    return mae


# **Helper for Load Data and Train Model**

**get_features**

In [7]:
!pip install -q mne

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 44.2 MB/s eta 0:00:00


In [8]:
import mne
def get_features(patient_metadata, recording_metadata, recording_data):
    # Extract features from the patient metadata.
    age = get_age(patient_metadata)
    sex = get_sex(patient_metadata)
    rosc = get_rosc(patient_metadata)
    ohca = get_ohca(patient_metadata)
    vfib = get_vfib(patient_metadata)
    ttm = get_ttm(patient_metadata)

    # Use one-hot encoding for sex; add more variables
    sex_features = np.zeros(2, dtype=int)
    if sex == 'Female':
        female = 1
        male   = 0
        other  = 0
    elif sex == 'Male':
        female = 0
        male   = 1
        other  = 0
    else:
        female = 0
        male   = 0
        other  = 1

    # Combine the patient features.
    patient_features = np.array([age, female, male, other, rosc, ohca, vfib, ttm])

    # Extract features from the recording data and metadata.
    channels = ['Fp1-F7', 'F7-T3', 'T3-T5', 'T5-O1', 'Fp2-F8', 'F8-T4', 'T4-T6', 'T6-O2', 'Fp1-F3',
                'F3-C3', 'C3-P3', 'P3-O1', 'Fp2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'Fz-Cz', 'Cz-Pz']
    num_channels = len(channels)
    num_recordings = len(recording_data)

    # Compute mean and standard deviation for each channel for each recording.
    available_signal_data = list()
    for i in range(num_recordings):
        signal_data, sampling_frequency, signal_channels = recording_data[i]
        if signal_data is not None:
            signal_data = reorder_recording_channels(signal_data, signal_channels, channels) # Reorder the channels in the signal data, as needed, for consistency across different recordings.
            available_signal_data.append(signal_data)

    if len(available_signal_data) > 0:
        available_signal_data = np.hstack(available_signal_data)
        signal_mean = np.nanmean(available_signal_data, axis=1)
        signal_std  = np.nanstd(available_signal_data, axis=1)
    else:
        signal_mean = float('nan') * np.ones(num_channels)
        signal_std  = float('nan') * np.ones(num_channels)

    # Compute the power spectral density for the delta, theta, alpha, and beta frequency bands for each channel of the most
    # recent recording.
    index = None
    for i in reversed(range(num_recordings)):
        signal_data, sampling_frequency, signal_channels = recording_data[i]
        if signal_data is not None:
            index = i
            break

    if index is not None:
        signal_data, sampling_frequency, signal_channels = recording_data[index]
        signal_data = reorder_recording_channels(signal_data, signal_channels, channels) # Reorder the channels in the signal data, as needed, for consistency across different recordings.

        delta_psd, _ = mne.time_frequency.psd_array_welch(signal_data, sfreq=sampling_frequency,  fmin=0.5,  fmax=8.0, verbose=False)
        theta_psd, _ = mne.time_frequency.psd_array_welch(signal_data, sfreq=sampling_frequency,  fmin=4.0,  fmax=8.0, verbose=False)
        alpha_psd, _ = mne.time_frequency.psd_array_welch(signal_data, sfreq=sampling_frequency,  fmin=8.0, fmax=12.0, verbose=False)
        beta_psd,  _ = mne.time_frequency.psd_array_welch(signal_data, sfreq=sampling_frequency, fmin=12.0, fmax=30.0, verbose=False)

        delta_psd_mean = np.nanmean(delta_psd, axis=1)
        theta_psd_mean = np.nanmean(theta_psd, axis=1)
        alpha_psd_mean = np.nanmean(alpha_psd, axis=1)
        beta_psd_mean  = np.nanmean(beta_psd,  axis=1)

        quality_score = get_quality_scores(recording_metadata)[index]
    else:
        delta_psd_mean = theta_psd_mean = alpha_psd_mean = beta_psd_mean = float('nan') * np.ones(num_channels)
        quality_score = float('nan')

    recording_features = np.hstack((signal_mean, signal_std, delta_psd_mean, theta_psd_mean, alpha_psd_mean, beta_psd_mean, quality_score))

    # Combine the features from the patient metadata and the recording data and metadata.
    features = np.hstack((patient_features, recording_features))

    return features

**save model**

In [9]:
import joblib
def save_challenge_model(model_folder, imputer, outcome_model, cpc_model):
    d = {'imputer': imputer, 'outcome_model': outcome_model, 'cpc_model': cpc_model}
    filename = os.path.join(model_folder, 'models.sav')
    joblib.dump(d, filename, protocol=0)

# **Helper for load model and Inference**

**Load Model**

In [10]:
# Load your trained models. This function is *required*. You should edit this function to add your code, but do *not* change the
# arguments of this function.
def load_challenge_models(model_folder, verbose):
    filename = os.path.join(model_folder, 'models.sav')
    return joblib.load(filename)

**run challenge model**

In [11]:
# Run your trained models. This function is *required*. You should edit this function to add your code, but do *not* change the
# arguments of this function.
def run_challenge_models(models, data_folder, patient_id, verbose):
    imputer = models['imputer']
    outcome_model = models['outcome_model']
    cpc_model = models['cpc_model']

    # Load data.
    patient_metadata, recording_metadata, recording_data = load_challenge_data(data_folder, patient_id)

    # Extract features.
    features = get_features(patient_metadata, recording_metadata, recording_data)
    features = features.reshape(1, -1)

    # Impute missing data.
    features = imputer.transform(features)

    # Apply models to features.
    outcome = outcome_model.predict(features)[0]
    outcome_probability = outcome_model.predict_proba(features)[0, 1]
    cpc = cpc_model.predict(features)[0]

    # Ensure that the CPC score is between (or equal to) 1 and 5.
    cpc = np.clip(cpc, 1, 5)

    return outcome, outcome_probability, cpc

# **Load Data and Train Model**

In [ ]:
# python train_model.py training_data model

In [12]:
data_folder = "/content/example/training_data"
model_folder = "/content/example/model"

In [13]:
print('Finding the Challenge data...')

patient_ids = find_data_folders(data_folder) #helper_code
num_patients = len(patient_ids)

if num_patients==0:
    raise FileNotFoundError('No data was provided.')

# Create a folder for the model if it does not already exist.
os.makedirs(model_folder, exist_ok=True)

# Extract the features and labels.

print('Extracting features and labels from the Challenge data...')

features = list()
outcomes = list()
cpcs = list()

for i in range(num_patients):
    print('    {}/{}...'.format(i+1, num_patients))

    # Load data.
    patient_id = patient_ids[i]
    patient_metadata, recording_metadata, recording_data = load_challenge_data(data_folder, patient_id) #helper_code

    # Extract features.
    current_features = get_features(patient_metadata, recording_metadata, recording_data) #team_code
    features.append(current_features)

    # Extract labels.
    current_outcome = get_outcome(patient_metadata)
    outcomes.append(current_outcome)
    current_cpc = get_cpc(patient_metadata)
    cpcs.append(current_cpc)

features = np.vstack(features) # (607, 117)
outcomes = np.vstack(outcomes) # (607, 1)
cpcs = np.vstack(cpcs) # (607, 1)

Finding the Challenge data...
Extracting features and labels from the Challenge data...
    1/13...
    2/13...
    3/13...
    4/13...
    5/13...
    6/13...
    7/13...
    8/13...
    9/13...
    10/13...
    11/13...
    12/13...
    13/13...


In [23]:
features[7]

array([82.,  1.,  0.,  0., nan,  1.,  1., 33., nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

In [21]:
outcomes[7].shape

(1,)

In [14]:
import xgboost as xgb
from sklearn.impute import SimpleImputer

In [15]:
# Train the models.
print('Training the Challenge models on the Challenge data...')

# Impute any missing features; use the mean value by default.
imputer = SimpleImputer().fit(features)

# Train the models.
features = imputer.transform(features)
outcome_model = xgb.XGBClassifier().fit(features, outcomes.ravel())
cpc_model = xgb.XGBRegressor().fit(features, cpcs.ravel())


Training the Challenge models on the Challenge data...


In [16]:
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

In [17]:
# Train the models.
print('Training the Challenge models on the Challenge data...')

# Define parameters for random forest classifier and regressor.
n_estimators   = 123  # Number of trees in the forest.
max_leaf_nodes = 456  # Maximum number of leaf nodes in each tree.
random_state   = 789  # Random state; set for reproducibility.

# Impute any missing features; use the mean value by default.
imputer = SimpleImputer().fit(features)

# Train the models.
features = imputer.transform(features)
outcome_model = RandomForestClassifier(
    n_estimators=n_estimators, max_leaf_nodes=max_leaf_nodes, random_state=random_state).fit(features, outcomes.ravel())
cpc_model = RandomForestRegressor(
    n_estimators=n_estimators, max_leaf_nodes=max_leaf_nodes, random_state=random_state).fit(features, cpcs.ravel())


Training the Challenge models on the Challenge data...


In [18]:
# Save the models.
save_challenge_model(model_folder, imputer, outcome_model, cpc_model) #team_code

# **Inference**

In [ ]:
# python run_model.py model test_data test_outputs

In [19]:
test_data = "/content/example/test_data" # input
test_outputs = "/content/example/test_outputs" # output

In [30]:
os.mkdir(test_data)

FileExistsError: ignored

In [ ]:
# making sample test_data
!cp -r /content/i-care-international-cardiac-arrest-research-consortium-database-1.0/training/ICARE_0284 "/content/test_data/ICARE_0284"
!cp -r /content/i-care-international-cardiac-arrest-research-consortium-database-1.0/training/ICARE_0286 "/content/test_data/ICARE_0286"
!cp -r /content/i-care-international-cardiac-arrest-research-consortium-database-1.0/training/ICARE_0296 "/content/test_data/ICARE_0296"
!cp -r /content/i-care-international-cardiac-arrest-research-consortium-database-1.0/training/ICARE_0299 "/content/test_data/ICARE_0299"

!cp -r /content/i-care-international-cardiac-arrest-research-consortium-database-1.0/training/ICARE_0462 "/content/test_data/ICARE_0462"
!cp -r /content/i-care-international-cardiac-arrest-research-consortium-database-1.0/training/ICARE_0467 "/content/test_data/ICARE_0467"
!cp -r /content/i-care-international-cardiac-arrest-research-consortium-database-1.0/training/ICARE_0469 "/content/test_data/ICARE_0469"
!cp -r /content/i-care-international-cardiac-arrest-research-consortium-database-1.0/training/ICARE_0471 "/content/test_data/ICARE_0471"
!cp -r /content/i-care-international-cardiac-arrest-research-consortium-database-1.0/training/ICARE_0472 "/content/test_data/ICARE_0472"
!cp -r /content/i-care-international-cardiac-arrest-research-consortium-database-1.0/training/ICARE_0481 "/content/test_data/ICARE_0481"
!cp -r /content/i-care-international-cardiac-arrest-research-consortium-database-1.0/training/ICARE_0483 "/content/test_data/ICARE_0483"
!cp -r /content/i-care-international-cardiac-arrest-research-consortium-database-1.0/training/ICARE_0514 "/content/test_data/ICARE_0514"
!cp -r /content/i-care-international-cardiac-arrest-research-consortium-database-1.0/training/ICARE_0520 "/content/test_data/ICARE_0520"

In [20]:
models = load_challenge_models(model_folder, True)

In [21]:
patient_ids = find_data_folders(test_data)
num_patients = len(patient_ids)

In [22]:
os.makedirs(test_outputs, exist_ok=True)

In [23]:
# Iterate over the patients.
for i in range(num_patients):

    print('    {}/{}...'.format(i+1, num_patients))

    patient_id = patient_ids[i]

    # Allow or disallow the models to fail on parts of the data; this can be helpful for debugging.
    try:
        outcome_binary, outcome_probability, cpc = run_challenge_models(models, test_data, patient_id, True) ### Teams: Implement this function!!!
    except:
        raise
        print('... failed.')
        outcome_binary, outcome_probability, cpc = float('nan'), float('nan'), float('nan')

    # Save Challenge outputs.

    # Create a folder for the Challenge outputs if it does not already exist.
    os.makedirs(os.path.join(test_outputs, patient_id), exist_ok=True)
    output_file = os.path.join(test_outputs, patient_id, patient_id + '.txt')
    save_challenge_outputs(output_file, patient_id, outcome_binary, outcome_probability, cpc)

    1/6...
    2/6...
    3/6...
    4/6...
    5/6...
    6/6...


In [60]:
!rm -r /content/example/test_data/.ipynb_checkpoints/

rm: cannot remove '/content/example/test_data/.ipynb_checkpoints/': No such file or directory


# **Evaluation**

In [ ]:
scores = evaluate_model(test_data, test_outputs)

# Unpack the scores.
challenge_score, auroc_outcomes, auprc_outcomes, accuracy_outcomes, f_measure_outcomes, mse_cpcs, mae_cpcs = scores

# Construct a string with scores.
output_string = \
    'Challenge Score: {:.3f}\n'.format(challenge_score) + \
    'Outcome AUROC: {:.3f}\n'.format(auroc_outcomes) + \
    'Outcome AUPRC: {:.3f}\n'.format(auprc_outcomes) + \
    'Outcome Accuracy: {:.3f}\n'.format(accuracy_outcomes) + \
    'Outcome F-measure: {:.3f}\n'.format(f_measure_outcomes) + \
    'CPC MSE: {:.3f}\n'.format(mse_cpcs) + \
    'CPC MAE: {:.3f}\n'.format(mae_cpcs)

print(output_string)


Challenge Score: 1.000
Outcome AUROC: 1.000
Outcome AUPRC: 1.000
Outcome Accuracy: 1.000
Outcome F-measure: 1.000
CPC MSE: 0.000
CPC MAE: 0.000



In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%ls

dataset.zip
drive/
i-care-international-cardiac-arrest-research-consortium-database-1.0/
model/
requirements.txt
sample_data/
test_data/
test_outputs/


In [ ]:
xgb.__version__

'1.7.4'

# **Submission**

In [ ]:
!cp /content/drive/MyDrive/Masoudi/2.zip .

In [ ]:
!unzip 2.zip

In [ ]:
%cd ~/

/root


In [ ]:
!mkdir example

In [ ]:
%cd example

/root/example


In [ ]:
!mkdir training_data test_data model test_outputs

In [ ]:
# put some data in test_data and training_data folders
!cp -r /content/training_data/ICARE_0284 /root/example/test_data
!cp -r /content/training_data/ICARE_0286 /root/example/test_data
!cp -r /content/training_data/ICARE_0296 /root/example/test_data
!cp -r /content/training_data/ICARE_0299 /root/example/test_data
!cp -r /content/training_data/ICARE_0303 /root/example/test_data

In [ ]:
!cp -r /content/training_data/ICARE_0303 /root/example/training_data
!cp -r /content/training_data/ICARE_0306 /root/example/training_data
!cp -r /content/training_data/ICARE_0311 /root/example/training_data
!cp -r /content/training_data/ICARE_0312 /root/example/training_data
!cp -r /content/training_data/ICARE_0313 /root/example/training_data
!cp -r /content/training_data/ICARE_0316 /root/example/training_data
!cp -r /content/training_data/ICARE_0319 /root/example/training_data

In [ ]:
!git clone https://github.com/physionetchallenges/python-example-2023.git

Cloning into 'python-example-2023'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 24 (delta 9), reused 21 (delta 7), pack-reused 0
Unpacking objects: 100% (24/24), 14.41 KiB | 1.60 MiB/s, done.


In [ ]:
!mv python-example-2023 python-classifier-2023

In [ ]:
#cp evaluatemodel.py to python-classifier-2023

In [ ]:
!ls

model  python-classifier-2023  test_data  test_outputs	training_data


In [ ]:
%cd /content/example/python-classifier-2023

/content/example/python-classifier-2023


In [ ]:
# Install Docker.
!apt-get -qq install docker.io

In [ ]:
docker run -it -v ~/example/model:/physionet/model -v ~/example/test_data:/physionet/test_data -v ~/example/test_outputs:/physionet/test_outputs -v ~/example/training_data:/physionet/training_data image bash
ls
pip install -r requirements.txt
python train_model.py training_data model
python run_model.py model test_data test_outputs
python evaluate_model.py test_data test_outputs

In [ ]:
!docker build -t image .

Cannot connect to the Docker daemon at unix:///var/run/docker.sock. Is the docker daemon running?


In [ ]:
%%shell
set -x
dockerd -b none --iptables=0 -l warn &
for i in $(seq 5); do [ ! -S "/var/run/docker.sock" ] && sleep 2 || break; done
docker build -t image .
docker run -it -v /content/example/model:/challenge/model -v /content/example/test_data:/challenge/test_data -v /content/example/test_outputs:/challenge/test_outputs -v /content/example/training_data:/challenge/training_data image bash
ls
python train_model.py training_data model
python run_model.py model test_data test_outputs
python evaluate_model.py test_data test_outputs
kill $(jobs -p)

+ dockerd -b none --iptables=0 -l warn
++ seq 5
+ for i in $(seq 5)
+ '[' '!' -S /var/run/docker.sock ']'
+ sleep 2
WARN[0000] containerd config version `1` has been deprecated and will be removed in containerd v2.0, please switch to version `2`, see https://github.com/containerd/containerd/blob/main/docs/PLUGINS.md#version-header 
WARN[2023-03-18T08:20:29.576848192Z] failed to load plugin io.containerd.snapshotter.v1.devmapper  error="devmapper not configured"
WARN[2023-03-18T08:20:29.577541489Z] could not use snapshotter devmapper in metadata plugin  error="devmapper not configured"
ERRO[2023-03-18T08:20:29.590968223Z] failed to initialize a tracing processor "otlp"  error="no OpenTelemetry endpoint: skip plugin"
ERRO[2023-03-18T08:20:29.615732927Z] failed to mount overlay: invalid argument     storage-driver=overlay2
ERRO[2023-03-18T08:20:29.616178118Z] exec: "fuse-overlayfs": executable file not found in $PATH  storage-driver=fuse-overlayfs
ERRO[2023-03-18T08:20:29.618304719Z] AUFS

In [ ]:
%%shell
set -x
dockerd -b none --iptables=0 -l warn &
for i in $(seq 5); do [ ! -S "/var/run/docker.sock" ] && sleep 2 || break; done
docker build -t image .
python train_model.py training_data model
python run_model.py model test_data test_outputs
kill $(jobs -p)

+ dockerd -b none --iptables=0 -l warn
++ seq 5
+ for i in $(seq 5)
+ '[' '!' -S /var/run/docker.sock ']'
+ sleep 2
WARN[0000] deprecated version : `1`, please switch to version `2` 
WARN[2023-03-04T14:25:20.768642525Z] failed to load plugin io.containerd.snapshotter.v1.devmapper  error="devmapper not configured"
WARN[2023-03-04T14:25:20.769273992Z] could not use snapshotter devmapper in metadata plugin  error="devmapper not configured"
ERRO[2023-03-04T14:25:20.808680528Z] failed to mount overlay: invalid argument     storage-driver=overlay2
ERRO[2023-03-04T14:25:20.809022848Z] exec: "fuse-overlayfs": executable file not found in $PATH  storage-driver=fuse-overlayfs
ERRO[2023-03-04T14:25:20.812041797Z] AUFS was not found in /proc/filesystems       storage-driver=aufs
ERRO[2023-03-04T14:25:20.824790025Z] failed to mount overlay: invalid argument     storage-driver=overlay
ERRO[2023-03-04T14:25:20.825328147Z] Udev sync is not supported. This will lead to data loss and unexpected behavior

In [ ]:
!docker build -t image .

Cannot connect to the Docker daemon at unix:///var/run/docker.sock. Is the docker daemon running?


## Installing Docker, Python 3 and TensorFlow

We will be running a linux machine, you can use any provider but for this tutorial we will be using Digital Ocean. For testing purposes we can just use a simple $5 per month [droplet](https://www.digitalocean.com/pricing/). We will be using the Ubuntu image from the droplet management panel.

Once the droplet has spun up with our Ubuntu image we will SSH into it to install the required packages. First we will intall Docker so we can download the pre installed PIP and python container. We are using Ubuntu 18.04 in this example.

First lets update all the packages to the latest ones with the following command.

In [ ]:
!sudo apt update

Reading package lists... Done
Building dependency tree       
Reading state information... Done
34 packages can be upgraded. Run 'apt list --upgradable' to see them.


Now we want to install some  prerequisite packages which will let us use HTTPS over apt.

In [ ]:
!sudo apt install apt-transport-https ca-certificates curl software-properties-common

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ca-certificates is already the newest version (20211016ubuntu0.20.04.1).
curl is already the newest version (7.68.0-1ubuntu2.16).
software-properties-common is already the newest version (0.99.9.11).
The following NEW packages will be installed:
  apt-transport-https
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 1,704 B of archives.
After this operation, 162 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 apt-transport-https all 2.0.9 [1,704 B]
Fetched 1,704 B in 0s (21.9 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This fronten

After that we will add the GPG key for the official Docker repository to the system.

In [ ]:
!curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo apt-key add -

OK


We will need to add the Docker repository to our APT sources:

In [ ]:
!sudo add-apt-repository "deb [arch=amd64] https://download.docker.com/linux/ubuntu bionic stable"

Hit:1 http://security.ubuntu.com/ubuntu focal-security InRelease
Get:2 https://download.docker.com/linux/ubuntu bionic InRelease [64.4 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Get:10 https://download.docker.com/linux/ubuntu bionic/stable amd64 Packages [39.2 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 104 kB in 2s (50.5 kB/s)
Reading package lists... Done


Next lets update the package database with our newly added Docker package repo.

In [ ]:
!sudo apt update

Hit:1 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:2 https://download.docker.com/linux/ubuntu bionic InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done
Building dependency tree       
Reading state information... Done
34 packages can be upgraded. Run 'apt list --upgradable' to see them

Finally lets install docker with the below command:

In [ ]:
!sudo apt install docker-ce

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  bridge-utils dns-root-data dnsmasq-base libidn11 netcat netcat-openbsd
  ubuntu-fan
Use 'sudo apt autoremove' to remove them.
The following additional packages will be installed:
  containerd.io docker-buildx-plugin docker-ce-cli docker-ce-rootless-extras
  docker-compose-plugin docker-scan-plugin slirp4netns
Suggested packages:
  aufs-tools cgroupfs-mount | cgroup-lite
The following packages will be REMOVED:
  containerd docker.io runc
The following NEW packages will be installed:
  containerd.io docker-buildx-plugin docker-ce docker-ce-cli
  docker-ce-rootless-extras docker-compose-plugin docker-scan-plugin
  slirp4netns
0 upgraded, 8 newly installed, 3 to remove and 34 not upgraded.
Need to get 112 MB of archives.
After this operation, 109 MB of additional disk space will be used.
Get:1 https://download.dock

Now lets check that docker is running. 

In [ ]:
!sudo systemctl status docker

System has not been booted with systemd as init system (PID 1). Can't operate.
Failed to connect to bus: Host is down


In [ ]:
!docker run \
  --runtime=nvidia \
  -it --rm -p 8081:8081 \
  -v "$PWD":/opt/colab \
  sorokine/docker-colab-local:latest

docker: Cannot connect to the Docker daemon at unix:///var/run/docker.sock. Is the docker daemon running?.
See 'docker run --help'.


The output from the command should look similar to the one below

In [ ]:
Output
● docker.service - Docker Application Container Engine
   Loaded: loaded (/lib/systemd/system/docker.service; enabled; vendor preset: enabled)
   Active: active (running) since Tue 2019-01-01 19:22:114 UTC; 1min 25s ago
     Docs: https://docs.docker.com
 Main PID: 10096 (dockerd)
    Tasks: 16
   CGroup: /system.slice/docker.service
           ├─10096 /usr/bin/dockerd -H fd://
           └─10113 docker-containerd --config /var/run/docker/containerd/containerd.toml

Once we have verified everything is installed and running correctly we can simply run the following command to install the TensorFlow Docker image:

In [ ]:
!docker run -it -p 8888:8888 gcr.io/tensorflow/tensorflow

docker: Cannot connect to the Docker daemon at unix:///var/run/docker.sock. Is the docker daemon running?.
See 'docker run --help'.


In [ ]:
!systemctl start docker

System has not been booted with systemd as init system (PID 1). Can't operate.
Failed to connect to bus: Host is down


Once installed we can move onto the next step.

In [ ]:
!zip -9 -r 2.zip /content/example

  adding: content/example/ (stored 0%)
  adding: content/example/model/ (stored 0%)
  adding: content/example/training_data/ (stored 0%)
  adding: content/example/training_data/ICARE_0313/ (stored 0%)
  adding: content/example/training_data/ICARE_0313/ICARE_0313_37.hea (deflated 74%)
  adding: content/example/training_data/ICARE_0313/ICARE_0313_42.hea (deflated 74%)
  adding: content/example/training_data/ICARE_0313/ICARE_0313_08.hea (deflated 74%)
  adding: content/example/training_data/ICARE_0313/ICARE_0313_29.hea (deflated 74%)
  adding: content/example/training_data/ICARE_0313/ICARE_0313_17.hea (deflated 74%)
  adding: content/example/training_data/ICARE_0313/ICARE_0313_20.mat (deflated 25%)
  adding: content/example/training_data/ICARE_0313/ICARE_0313_27.mat (deflated 24%)
  adding: content/example/training_data/ICARE_0313/ICARE_0313_31.mat (deflated 22%)
  adding: content/example/training_data/ICARE_0313/RECORDS (deflated 82%)
  adding: content/example/training_data/ICARE_0313/IC

In [ ]:
!ls -l

total 18588564
-rw-r--r--   1 root root   402492811 Mar 17 17:18 2.zip
-rw-------   1 root root 18632148751 Mar 17 15:06 dataset.zip
drwx------   5 root root        4096 Mar 17 15:03 drive
drwxr-xr-x   7 root root        4096 Mar 17 15:58 example
drwxr-xr-x   1 root root        4096 Mar 15 13:40 sample_data
drwxr-xr-x 609 root root       20480 Mar 17 15:19 training_data


In [ ]:
%cd ..

/content


In [ ]:
!cp /content/2.zip /content/drive/MyDrive/Masoudi

In [ ]:
# Mount drive

In [ ]:
!cp /content/drive/MyDrive/Masoudi/dataset.zip .

In [ ]:
!unzip -qq dataset.zip

In [ ]:
!mkdir training_data

In [ ]:
!mv -v /content/i-care-international-cardiac-arrest-research-consortium-database-1.0/training/* training_data

In [ ]:
!rm -r /content/i-care-international-cardiac-arrest-research-consortium-database-1.0